
<a id='whatsnew-0151'></a>

# v0.15.1 (November 9, 2014)

{{ header }}

This is a minor bug-fix release from 0.15.0 and includes a small number of API changes, several new features,
enhancements, and performance improvements along with a large number of bug fixes. We recommend that all
users upgrade to this version.

- [Enhancements](#whatsnew-0151-enhancements)  
- [API Changes](#whatsnew-0151-api)  
- [Bug Fixes](#whatsnew-0151-bug-fixes)  



<a id='whatsnew-0151-api'></a>

## API changes

- `s.dt.hour` and other `.dt` accessors will now return `np.nan` for missing values (rather than previously -1), ([GH8689](https://github.com/pandas-dev/pandas/issues/8689))  
previous behavior:  

```ipython
In [6]: s.dt.hour
Out[6]:
0    0
1    0
2   -1
3    0
4    0
dtype: int64
```


current behavior:  
- `groupby` with `as_index=False` will not add erroneous extra columns to
result ([GH8582](https://github.com/pandas-dev/pandas/issues/8582)):  
previous behavior:  

```ipython
In [4]: df.groupby(ts, as_index=False).max()
Out[4]:
   NaN  jim  joe
0    0   72   83
1    5   77   84
2   10   96   65
```


current behavior:  
- `groupby` will not erroneously exclude columns if the column name conflicts
with the grouper name ([GH8112](https://github.com/pandas-dev/pandas/issues/8112)):  
previous behavior (excludes 1st column from output):  

```ipython
In [4]: gr.apply(sum)
Out[4]:
       joe
jim
False   24
True    11
```


current behavior:  
- Support for slicing with monotonic decreasing indexes, even if `start` or `stop` is
not found in the index ([GH7860](https://github.com/pandas-dev/pandas/issues/7860)):  
previous behavior:  

```ipython
In [8]: s.loc[3.5:1.5]
KeyError: 3.5
```


current behavior:  
- `io.data.Options` has been fixed for a change in the format of the Yahoo Options page ([GH8612](https://github.com/pandas-dev/pandas/issues/8612)), ([GH8741](https://github.com/pandas-dev/pandas/issues/8741))  
>**Note**
>
>As a result of a change in Yahoo’s option page layout, when an expiry date is given,
`Options` methods now return data for a single expiry date.  Previously, methods returned all
data for the selected month.  
The `month` and `year` parameters have been undeprecated and can be used to get all
options data for a given month.  
If an expiry date that is not valid is given, data for the next expiry after the given
date is returned.  
Option data frames are now saved on the instance as `callsYYMMDD` or `putsYYMMDD`.  Previously
they were saved as `callsMMYY` and `putsMMYY`.  The next expiry is saved as `calls` and `puts`.  
New features:  
- The expiry parameter can now be a single date or a list-like object containing dates.  
- A new property `expiry_dates` was added, which returns all available expiry dates.  
Current behavior:  

```ipython
In [17]: from pandas.io.data import Options

In [18]: aapl = Options('aapl', 'yahoo')

In [19]: aapl.get_call_data().iloc[0:5, 0:1]
Out[19]:
                                             Last
Strike Expiry     Type Symbol
80     2014-11-14 call AAPL141114C00080000  29.05
84     2014-11-14 call AAPL141114C00084000  24.80
85     2014-11-14 call AAPL141114C00085000  24.05
86     2014-11-14 call AAPL141114C00086000  22.76
87     2014-11-14 call AAPL141114C00087000  21.74

In [20]: aapl.expiry_dates
Out[20]:
[datetime.date(2014, 11, 14),
 datetime.date(2014, 11, 22),
 datetime.date(2014, 11, 28),
 datetime.date(2014, 12, 5),
 datetime.date(2014, 12, 12),
 datetime.date(2014, 12, 20),
 datetime.date(2015, 1, 17),
 datetime.date(2015, 2, 20),
 datetime.date(2015, 4, 17),
 datetime.date(2015, 7, 17),
 datetime.date(2016, 1, 15),
 datetime.date(2017, 1, 20)]

In [21]: aapl.get_near_stock_price(expiry=aapl.expiry_dates[0:3]).iloc[0:5, 0:1]
Out[21]:
                                            Last
Strike Expiry     Type Symbol
109    2014-11-22 call AAPL141122C00109000  1.48
       2014-11-28 call AAPL141128C00109000  1.79
110    2014-11-14 call AAPL141114C00110000  0.55
       2014-11-22 call AAPL141122C00110000  1.02
       2014-11-28 call AAPL141128C00110000  1.32
```





<a id='whatsnew-0151-datetime64-plotting'></a>
- pandas now also registers the `datetime64` dtype in matplotlib’s units registry
  to plot such values as datetimes. This is activated once pandas is imported. In
  previous versions, plotting an array of `datetime64` values will have resulted
  in plotted integer values. To keep the previous behaviour, you can do
  `del matplotlib.units.registry[np.datetime64]` ([GH8614](https://github.com/pandas-dev/pandas/issues/8614)).  



<a id='whatsnew-0151-enhancements'></a>

## Enhancements

- `concat` permits a wider variety of iterables of pandas objects to be
passed as the first parameter ([GH8645](https://github.com/pandas-dev/pandas/issues/8645)):  
previous behavior:  

```ipython
In [7]: pd.concat(deque((df1, df2)))
TypeError: first argument must be a list-like of pandas objects, you passed an object of type "deque"
```


current behavior:  
- Represent `MultiIndex` labels with a dtype that utilizes memory based on the level size. In prior versions, the memory usage was a constant 8 bytes per element in each level. In addition, in prior versions, the *reported* memory usage was incorrect as it didn’t show the usage for the memory occupied by the underling data array. ([GH8456](https://github.com/pandas-dev/pandas/issues/8456))  
previous behavior:  

```ipython
# this was underreported in prior versions
In [1]: dfi.memory_usage(index=True)
Out[1]:
Index    8000 # took about 24008 bytes in < 0.15.1
A        8000
dtype: int64
```


current behavior:  
- Added Index properties is_monotonic_increasing and is_monotonic_decreasing ([GH8680](https://github.com/pandas-dev/pandas/issues/8680)).  
- Added option to select columns when importing Stata files ([GH7935](https://github.com/pandas-dev/pandas/issues/7935))  
- Qualify memory usage in `DataFrame.info()` by adding `+` if it is a lower bound ([GH8578](https://github.com/pandas-dev/pandas/issues/8578))  
- Raise errors in certain aggregation cases where an argument such as `numeric_only` is not handled ([GH8592](https://github.com/pandas-dev/pandas/issues/8592)).  
- Added support for 3-character ISO and non-standard country codes in `io.wb.download()` ([GH8482](https://github.com/pandas-dev/pandas/issues/8482))  
- World Bank data requests now will warn/raise based
  on an `errors` argument, as well as a list of hard-coded country codes and
  the World Bank’s JSON response.  In prior versions, the error messages
  didn’t look at the World Bank’s JSON response.  Problem-inducing input were
  simply dropped prior to the request. The issue was that many good countries
  were cropped in the hard-coded approach.  All countries will work now, but
  some bad countries will raise exceptions because some edge cases break the
  entire response. ([GH8482](https://github.com/pandas-dev/pandas/issues/8482))  
- Added option to `Series.str.split()` to return a `DataFrame` rather than a `Series` ([GH8428](https://github.com/pandas-dev/pandas/issues/8428))  
- Added option to `df.info(null_counts=None|True|False)` to override the default display options and force showing of the null-counts ([GH8701](https://github.com/pandas-dev/pandas/issues/8701))  



<a id='whatsnew-0151-bug-fixes'></a>

## Bug fixes

- Bug in unpickling of a `CustomBusinessDay` object ([GH8591](https://github.com/pandas-dev/pandas/issues/8591))  
- Bug in coercing `Categorical` to a records array, e.g. `df.to_records()` ([GH8626](https://github.com/pandas-dev/pandas/issues/8626))  
- Bug in `Categorical` not created properly with `Series.to_frame()` ([GH8626](https://github.com/pandas-dev/pandas/issues/8626))  
- Bug in coercing in astype of a `Categorical` of a passed `pd.Categorical` (this now raises `TypeError` correctly), ([GH8626](https://github.com/pandas-dev/pandas/issues/8626))  
- Bug in `cut`/`qcut` when using `Series` and `retbins=True` ([GH8589](https://github.com/pandas-dev/pandas/issues/8589))  
- Bug in writing Categorical columns to an SQL database with `to_sql` ([GH8624](https://github.com/pandas-dev/pandas/issues/8624)).  
- Bug in comparing `Categorical` of datetime raising when being compared to a scalar datetime ([GH8687](https://github.com/pandas-dev/pandas/issues/8687))  
- Bug in selecting from a `Categorical` with `.iloc` ([GH8623](https://github.com/pandas-dev/pandas/issues/8623))  
- Bug in groupby-transform with a Categorical ([GH8623](https://github.com/pandas-dev/pandas/issues/8623))  
- Bug in duplicated/drop_duplicates with a Categorical ([GH8623](https://github.com/pandas-dev/pandas/issues/8623))  
- Bug in `Categorical` reflected comparison operator raising if the first argument was a numpy array scalar (e.g. np.int64) ([GH8658](https://github.com/pandas-dev/pandas/issues/8658))  
- Bug in Panel indexing with a list-like ([GH8710](https://github.com/pandas-dev/pandas/issues/8710))  
- Compat issue is `DataFrame.dtypes` when `options.mode.use_inf_as_null` is True ([GH8722](https://github.com/pandas-dev/pandas/issues/8722))  
- Bug in `read_csv`, `dialect` parameter would not take a string ([GH8703](https://github.com/pandas-dev/pandas/issues/8703))  
- Bug in slicing a MultiIndex level with an empty-list ([GH8737](https://github.com/pandas-dev/pandas/issues/8737))  
- Bug in numeric index operations of add/sub with Float/Index Index with numpy arrays ([GH8608](https://github.com/pandas-dev/pandas/issues/8608))  
- Bug in setitem with empty indexer and unwanted coercion of dtypes ([GH8669](https://github.com/pandas-dev/pandas/issues/8669))  
- Bug in ix/loc block splitting on setitem (manifests with integer-like dtypes, e.g. datetime64) ([GH8607](https://github.com/pandas-dev/pandas/issues/8607))  
- Bug when doing label based indexing with integers not found in the index for
  non-unique but monotonic indexes ([GH8680](https://github.com/pandas-dev/pandas/issues/8680)).  
- Bug when indexing a Float64Index with `np.nan` on numpy 1.7 ([GH8980](https://github.com/pandas-dev/pandas/issues/8980)).  
- Fix `shape` attribute for `MultiIndex` ([GH8609](https://github.com/pandas-dev/pandas/issues/8609))  
- Bug in `GroupBy` where a name conflict between the grouper and columns
  would break `groupby` operations ([GH7115](https://github.com/pandas-dev/pandas/issues/7115), [GH8112](https://github.com/pandas-dev/pandas/issues/8112))  
- Fixed a bug where plotting a column `y` and specifying a label would mutate the index name of the original DataFrame ([GH8494](https://github.com/pandas-dev/pandas/issues/8494))  
- Fix regression in plotting of a DatetimeIndex directly with matplotlib ([GH8614](https://github.com/pandas-dev/pandas/issues/8614)).  
- Bug in `date_range` where partially-specified dates would incorporate current date ([GH6961](https://github.com/pandas-dev/pandas/issues/6961))  
- Bug in Setting by indexer to a scalar value with a mixed-dtype Panel4d was failing ([GH8702](https://github.com/pandas-dev/pandas/issues/8702))  
- Bug where `DataReader`’s would fail if one of the symbols passed was invalid.  Now returns data for valid symbols and np.nan for invalid ([GH8494](https://github.com/pandas-dev/pandas/issues/8494))  
- Bug in `get_quote_yahoo` that wouldn’t allow non-float return values ([GH5229](https://github.com/pandas-dev/pandas/issues/5229)).  



<a id='whatsnew-0-15-1-contributors'></a>

## Contributors

A total of 23 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- Aaron Staple +  
- Andrew Rosenfeld  
- Anton I. Sipos  
- Artemy Kolchinsky  
- Bill Letson +  
- Dave Hughes +  
- David Stephens  
- Guillaume Horel +  
- Jeff Reback  
- Joris Van den Bossche  
- Kevin Sheppard  
- Nick Stahl +  
- Sanghee Kim +  
- Stephan Hoyer  
- Tom Augspurger  
- TomAugspurger  
- WANG Aiyong +  
- behzad nouri  
- immerrr  
- jnmclarty  
- jreback  
- pallav-fdsi +  
- unutbu  